In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import cv2
import numpy as np


In [4]:
# 경로지정
answer_path = '/content/drive/MyDrive/GDSC_AI/BARO/source/answer_jpg'
wrong_path = '/content/drive/MyDrive/GDSC_AI/BARO/source/wrong_jpg'

In [5]:
## 전처리

def load_and_preprocess_images(folder_path, label, img_size=(28, 28)):
    images = []
    labels = []

    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # 정규화
        images.append(img)
        labels.append(label)

    return images, labels

In [6]:
# label 할당(1값: 정상, 0값: 거북이)

answer_images, answer_labels = load_and_preprocess_images(answer_path, label=1)
wrong_images, wrong_labels = load_and_preprocess_images(wrong_path, label=0)

In [7]:
images = np.array(answer_images + wrong_images)
labels = np.array(answer_labels + wrong_labels)

In [8]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
model.fit(images, labels, epochs = 15)

Epoch 1/15
10/10 [==============================] - 2s 30ms/step - loss: 0.6582 - accuracy: 0.6471
Epoch 2/15
10/10 [==============================] - 0s 29ms/step - loss: 0.6251 - accuracy: 0.6609
Epoch 3/15
10/10 [==============================] - 0s 27ms/step - loss: 0.5750 - accuracy: 0.7232
Epoch 4/15
10/10 [==============================] - 0s 29ms/step - loss: 0.4617 - accuracy: 0.8547
Epoch 5/15
10/10 [==============================] - 0s 27ms/step - loss: 0.3781 - accuracy: 0.8858
Epoch 6/15
10/10 [==============================] - 0s 31ms/step - loss: 0.3312 - accuracy: 0.8997
Epoch 7/15
10/10 [==============================] - 0s 27ms/step - loss: 0.2408 - accuracy: 0.9100
Epoch 8/15
10/10 [==============================] - 0s 29ms/step - loss: 0.1988 - accuracy: 0.9273
Epoch 9/15
10/10 [==============================] - 0s 26ms/step - loss: 0.2033 - accuracy: 0.9204
Epoch 10/15
10/10 [==============================] - 0s 26ms/step - loss: 0.1632 - accuracy: 0.9377
Epoch 11/

In [11]:
model.save("CNN_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
model_tfl = tf.keras.models.load_model('/content/CNN_model.h5')

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tfl)
tflite_model = converter.convert()

In [14]:
with open('CNN_model.tflite', 'wb') as f:
  f.write(tflite_model)